In [2]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv("satellite_S1_enrichi_part.csv", dtype={"chemin_directory": str})
print(df[df["chemin_directory"] == "1"]["chemin_directory"].head(10))#remettre le fichier csv avec ce format
df.to_csv("satellite_S1_new.csv", index=False)

332     1
448     1
474     1
942     1
1105    1
1286    1
1770    1
1825    1
2345    1
2411    1
Name: chemin_directory, dtype: object


In [60]:
import re
import pandas as pd
import csv

def extract_directory_from_id(label_info):
    """
    Extrait le chiffre après 'labels_' dans une chaîne de caractères.
    Préserve les zéros non significatifs.
    
    Exemples:
      'sen12floods_s1_labels_0026_2018_12_19' -> '0026'
      'sen12floods_s1_labels_26_2019_01_23'  -> '26'
    """
    if pd.isna(label_info):  # Gestion des valeurs NaN
        return None
        
    match = re.search(r'labels_(\d+)', str(label_info))
    if match:
        return match.group(1)  # Retourne les chiffres extraits en tant que chaîne
    return None

# Application de la fonction à la colonne 'id'
df["chemin_directory"] = df["id"].apply(extract_directory_from_id)
df["chemin_directory"] = df["chemin_directory"].apply(lambda x: f"'{x}" if pd.notna(x) else x)

# Afficher quelques résultats pour vérification
print(df[["id", "chemin_directory"]].head(10))
df.to_csv("satellite_S1_new.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

                                      id chemin_directory
0  sen12floods_s1_labels_0165_2019_02_06            '0165
1  sen12floods_s1_labels_0026_2018_12_19            '0026
2  sen12floods_s1_labels_0100_2018_12_17            '0100
3  sen12floods_s1_labels_0305_2019_02_11            '0305
4  sen12floods_s1_labels_0246_2019_04_06            '0246
5  sen12floods_s1_labels_0272_2019_03_09            '0272
6  sen12floods_s1_labels_0214_2019_02_28            '0214
7  sen12floods_s1_labels_0321_2019_03_07            '0321
8  sen12floods_s1_labels_0143_2019_02_06            '0143
9  sen12floods_s1_labels_0320_2019_02_23            '0320


In [54]:
extract_directory_from_id("sen12floods_s1_labels_0026_2018_12_19")  # Doit retourner '0026'

'0026'

In [ ]:
"""propriétés des images 
Géoréférencement : Système de projection, coordonnées, résolution
Spécifications des bandes : Plage de longueurs d’onde, sensibilité, numéros de bandes
Angles solaires : Zenith et azimut, utiles pour corriger l’éclairage

Bandes : VV (vertical-vertical) et VH (vertical-horizontal)
Résolution : ~10 m
Avantage : Pénètre les nuages, top pour surveiller les inondations
Sentinel-2 (Optique multispectrale)

Format : GeoTIFF
Bandes : 12 bandes (certaines à 10 m, d’autres à 20 m voire 60 m)

NDVI = (Bande NIR – Bande Rouge) / (Bande NIR + Bande Rouge)

Pourquoi NDVI ? : Il te donne une indication sur la vitalité de la végétation.
"""

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import cv2
from tqdm import tqdm

ModuleNotFoundError: No module named 'torch'

In [ ]:
def load_metadata(csv_path):
    """
    Load and preprocess the metadata CSV file
    """
    df = pd.read_csv(csv_path)
    
    if not pd.api.types.is_datetime64_any_dtype(df['date']):
        df['date'] = pd.to_datetime(df['date'])
        
    #nettoyage du nouveau_chemin comme présent dans le fichier csv actuelleemnt , les 
    df['nouveau_chemin'] = df['nouveau_chemin'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    
    #"dans nouveau chemin , on peut recréer le chemin de l'image , avec la polarisation manière plus précise"
    df['vh_path'] = df['nouveau_chemin'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
    df['vv_path'] = df['nouveau_chemin'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
    
    print(f"Loaded {len(df)} samples with {df['label'].sum()} flood images")
    print(f"Class distribution: {df['label'].value_counts(normalize=True).to_dict()}")
    
    return df

# Load the dataset
data_df = load_metadata('satellite_s1_refontes.csv')

# Preview data
print(data_df.head())

In [ ]:
class SARFloodDataset(Dataset):
    """
    Custom Dataset for loading SAR (Sentinel-1) flood detection data
    """
    def __init__(self, dataframe, img_size=(256, 256), transform=None, test_mode=False):
        """
        Args:
            dataframe: Pandas dataframe containing image paths and labels
            img_size: Target size for resizing images
            transform: Optional transformations to apply
            test_mode: If True, return image paths along with data
        """
        self.dataframe = dataframe
        self.img_size = img_size
        self.transform = transform
        self.test_mode = test_mode
        
        # Check if all paths exist
        self._validate_paths()
        
    def _validate_paths(self):
        """Validate if image paths exist"""
        valid_indices = []
        for idx, row in tqdm(self.dataframe.iterrows(), desc="Validating paths", total=len(self.dataframe)):
            if os.path.exists(row['vh_path']) and os.path.exists(row['vv_path']):
                valid_indices.append(idx)
        
        print(f"Valid paths: {len(valid_indices)}/{len(self.dataframe)}")
        self.dataframe = self.dataframe.loc[valid_indices].reset_index(drop=True)
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        """
        Load and return a sample from the dataset
        """
        # Get image paths and label
        vh_path = self.dataframe.iloc[idx]['vh_path']
        vv_path = self.dataframe.iloc[idx]['vv_path']
        label = self.dataframe.iloc[idx]['label']
        
        # Load SAR images (VH and VV channels)
        vh_image = self._load_raster(vh_path, scaling_value=50.0)  # Different scaling for VH
        vv_image = self._load_raster(vv_path, scaling_value=100.0)  # Different scaling for VV
        
        # Stack channels to create a 2-channel image
        image = np.stack([vh_image, vv_image], axis=0)  # Shape: (2, H, W)
        
        # Convert to tensor
        image = torch.from_numpy(image).float()
        
        # Apply transformations if specified
        if self.transform:
            image = self.transform(image)
        
        # Return image path for testing/visualization if in test mode
        if self.test_mode:
            return image, label, {'vh_path': vh_path, 'vv_path': vv_path}
        
        return image, label
        
    def _load_raster(self, filepath, scaling_value=1.0):
        """
        Load a single band raster file and preprocess it
        
        Args:
            filepath: Path to the raster file
            scaling_value: Value to scale the data by
            
        Returns:
            Preprocessed raster data
        """
        try:
            with rasterio.open(filepath) as src:
                # Read the raster data and squeeze to remove singleton dimensions
                band = src.read(1)
                
                # Apply scaling
                band = band / scaling_value
                
                # Clip values to prevent outliers
                band = np.clip(band, 0, 5)
                
                # Resize to target size
                band = cv2.resize(band, self.img_size)
                
                return band
        except Exception as e:
            print(f"Error loading {filepath}: {e}")
            # Return a zero array if loading fails
            return np.zeros(self.img_size)

In [ ]:
def create_dataloaders(dataframe, img_size=(256, 256), batch_size=32, test_size=0.2, random_state=42):
    """
    Split the data and create DataLoaders for training and validation
    """
    # Split data into train and validation sets
    train_df, val_df = train_test_split(
        dataframe, 
        test_size=test_size, 
        random_state=random_state,
        stratify=dataframe['label']  # Ensure class balance in both sets
    )
    
    print(f"Training set: {len(train_df)} samples")
    print(f"Validation set: {len(val_df)} samples")
    
    # Define transformations for training and validation
    # For SAR data, we need to be careful with augmentations
    # Certain transformations like horizontal/vertical flips can change the physical meaning
    train_transform = transforms.Compose([
        # RandomCrop can be safe for SAR data
        transforms.RandomCrop(size=img_size, padding=int(img_size[0]*0.1)),
        # Subtle rotation (small angles)
        transforms.RandomRotation(degrees=10),
        # Normalize the data - values determined empirically from your dataset
        transforms.Normalize(mean=[0.5, 0.5], std=[0.5, 0.5])  # For 2-channel data
    ])
    
    val_transform = transforms.Compose([
        # Only normalize for validation
        transforms.Normalize(mean=[0.5, 0.5], std=[0.5, 0.5])
    ])
    
    # Create datasets
    train_dataset = SARFloodDataset(train_df, img_size=img_size, transform=train_transform)
    val_dataset = SARFloodDataset(val_df, img_size=img_size, transform=val_transform)
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    return train_loader, val_loader

# Create dataloaders
train_loader, val_loader = create_dataloaders(
    data_df,
    img_size=(256, 256),
    batch_size=32
)

In [ ]:
def visualize_sar_images(dataloader, num_samples=3):
    """
    Visualize SAR images from the dataloader
    """
    # Get a batch of data
    images, labels = next(iter(dataloader))
    
    # Select random samples
    indices = np.random.choice(range(len(images)), num_samples, replace=False)
    
    fig, axes = plt.subplots(num_samples, 2, figsize=(10, 5*num_samples))
    
    for i, idx in enumerate(indices):
        img = images[idx].numpy()  # Convert to numpy
        label = labels[idx].item()  # Get the label
        
        # VH channel
        axes[i, 0].imshow(img[0], cmap='gray')
        axes[i, 0].set_title(f"VH Channel - {'Flood' if label==1 else 'No Flood'}")
        axes[i, 0].axis('off')
        
        # VV channel
        axes[i, 1].imshow(img[1], cmap='gray')
        axes[i, 1].set_title(f"VV Channel - {'Flood' if label==1 else 'No Flood'}")
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize some samples
visualize_sar_images(train_loader)

In [ ]:
class SARFloodNet(nn.Module):
    """
    CNN for SAR-based flood detection with 2 input channels (VH, VV)
    """
    def __init__(self, num_classes=2):
        super(SARFloodNet, self).__init__()
        
        # Input: 2 channels (VH and VV polarization)
        self.features = nn.Sequential(
            # First conv block
            # Input: 2×256×256, Output: 64×128×128
            nn.Conv2d(2, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Second conv block
            # Input: 64×128×128, Output: 128×64×64
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Third conv block
            # Input: 128×64×64, Output: 256×32×32
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Fourth conv block
            # Input: 256×32×32, Output: 512×16×16
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Fifth conv block
            # Input: 512×16×16, Output: 512×8×8
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Adaptive pooling to handle variable input sizes
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Fully connected layers for classification
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),  # Regularization
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        # Apply convolutional feature extraction
        x = self.features(x)
        
        # Apply adaptive pooling
        x = self.adaptive_pool(x)
        
        # Flatten for fully connected layers
        x = torch.flatten(x, 1)
        
        # Apply classifier
        x = self.classifier(x)
        
        return x

# Initialize the model
model = SARFloodNet(num_classes=2)
print(model)

# Count the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of trainable parameters: {count_parameters(model):,}")

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs=20, learning_rate=0.001, 
                device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Train the model and validate
    """
    print(f"Training on {device}")
    model.to(device)
    
    # Define loss function and optimizer
    # Using CrossEntropyLoss for multi-class classification
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 'min', patience=3, factor=0.5, verbose=True
    )
    
    # For tracking best performance
    best_val_loss = float('inf')
    best_model_state = None
    
    # Training history
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': [],
        'precision': [],
        'recall': [],
        'f1_score': []
    }
    
    # Training loop
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        for images, labels in train_progress:
            # Move tensors to device
            images, labels = images.to(device), labels.to(device)
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Track statistics
            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            # Update progress bar
            train_progress.set_postfix({'loss': loss.item(), 'acc': train_correct/train_total})
        
        # Calculate epoch statistics
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        # For metrics
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            val_progress = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")
            for images, labels in val_progress:
                # Move tensors to device
                images, labels = images.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                # Track statistics
                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
                # Store predictions and labels for metrics
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                
                # Update progress bar
                val_progress.set_postfix({'loss': loss.item(), 'acc': val_correct/val_total})
        
        # Calculate epoch statistics
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_correct / val_total
        
        # Calculate metrics
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=0)
        recall = recall_score(all_labels, all_preds, average='binary', zero_division=0)
        f1 = f1_score(all_labels, all_preds, average='binary', zero_division=0)
        
        # Update learning rate
        scheduler.step(val_loss)
        
        # Print epoch statistics
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
            print("Saved new best model!")
        
        # Save history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['precision'].append(precision)
        history['recall'].append(recall)
        history['f1_score'].append(f1)
    
    # Load best model
    model.load_state_dict(best_model_state)
    
    return model, history

# Train the model
trained_model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=20,
    learning_rate=0.001
)

# Save the trained model
torch.save(trained_model.state_dict(), 'sar_flood_detection_model.pth')

In [ ]:
def plot_training_history(history):
    """
    Plot training history
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot loss
    axes[0, 0].plot(history['train_loss'], label='Train Loss')
    axes[0, 0].plot(history['val_loss'], label='Validation Loss')
    axes[0, 0].set_title('Loss')
    axes[0, 0].set_xlabel('Epochs')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    
    # Plot accuracy
    axes[0, 1].plot(history['train_acc'], label='Train Accuracy')
    axes[0, 1].plot(history['val_acc'], label='Validation Accuracy')
    axes[0, 1].set_title('Accuracy')
    axes[0, 1].set_xlabel('Epochs')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    
    # Plot precision and recall
    axes[1, 0].plot(history['precision'], label='Precision')
    axes[1, 0].plot(history['recall'], label='Recall')
    axes[1, 0].set_title('Precision and Recall')
    axes[1, 0].set_xlabel('Epochs')
    axes[1, 0].set_ylabel('Score')
    axes[1, 0].legend()
    
    # Plot F1 score
    axes[1, 1].plot(history['f1_score'], label='F1 Score')
    axes[1, 1].set_title('F1 Score')
    axes[1, 1].set_xlabel('Epochs')
    axes[1, 1].set_ylabel('Score')
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()

# Plot the training history
plot_training_history(history)

In [ ]:
def evaluate_model(model, test_loader, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Evaluate the model on the test set
    """
    model.to(device)
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='binary', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='binary', zero_division=0)
    
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1-Score: {f1:.4f}")
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

def predict_flood(model, image_vh_path, image_vv_path, img_size=(256, 256), 
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Make a prediction on a single SAR image
    """
    model.to(device)
    model.eval()
    
    # Load the image
    vh_image = load_and_preprocess_sar(image_vh_path, scaling_value=50.0, img_size=img_size)
    vv_image = load_and_preprocess_sar(image_vv_path, scaling_value=100.0, img_size=img_size)
    
    # Stack channels
    image = np.stack([vh_image, vv_image], axis=0)
    
    # Convert to tensor
    image = torch.from_numpy(image).float().unsqueeze(0)  # Add batch dimension
    
    # Normalize
    image = transforms.Normalize(mean=[0.5, 0.5], std=[0.5, 0.5])(image)
    
    # Move to device
    image = image.to(device)
    
    # Make prediction
    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        flood_probability = probabilities[0, 1].item()  # Probability of class 1 (flood)
    
    return {
        'predicted_class': predicted_class,  # 0: No Flood, 1: Flood
        'flood_probability': flood_probability,
        'prediction': 'Flood' if predicted_class == 1 else 'No Flood'
    }

def load_and_preprocess_sar(filepath, scaling_value=1.0, img_size=(256, 256)):
    """
    Load and preprocess a single SAR image
    """
    with rasterio.open(filepath) as src:
        band = src.read(1)
        
        # Apply scaling
        band = band / scaling_value
        
        # Clip values
        band = np.clip(band, 0, 5)
        
        # Resize
        band = cv2.resize(band, img_size)
        
        return band

# Example of making a prediction
def visualize_prediction(model, test_loader, num_samples=3, 
                        device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Visualize model predictions on random test samples
    """
    model.to(device)
    model.eval()
    
    # Get a batch of data
    dataiter = iter(test_loader)
    images, labels = next(dataiter)
    
    # Select random samples
    indices = np.random.choice(range(len(images)), num_samples, replace=False)
    
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 4*num_samples))
    
    for i, idx in enumerate(indices):
        img = images[idx].unsqueeze(0).to(device)  # Add batch dimension and move to device
        label = labels[idx].item()
        
        # Make prediction
        with torch.no_grad():
            outputs = model(img)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            flood_probability = probabilities[0, 1].item()
        
        # Plot VH channel
        axes[i, 0].imshow(images[idx][0].numpy(), cmap='gray')
        axes[i, 0].set_title(f"VH Channel - True: {'Flood' if label==1 else 'No Flood'}")
        axes[i, 0].axis('off')
        
        # Plot VV channel
        axes[i, 1].imshow(images[idx][1].numpy(), cmap='gray')
        title = f"VV Channel - Pred: {'Flood' if predicted_class==1 else 'No Flood'} ({flood_probability:.2f})"
        axes[i, 1].set_title(title, color='green' if predicted_class == label else 'red')
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Create a test loader (if you have separate test data)
# For demonstration, we'll use the validation loader
test_loader = val_loader

# Evaluate the model
metrics = evaluate_model(trained_model, test_loader)

# Visualize some predictions
visualize_prediction(trained_model, test_loader, num_samples=5)